NOTE : Considering some of GEE's user limits and avoiding some system errors, such as "ERROR: user memory limit exceeded","TimeoutError", etc,we will process or download some data beforehand and upload it to GEE Assets, where it will be called up when needed.

Special reminder: the following variables or parameters need to be modified

Must be amended : "roi", "region", "elevation", "dataset_id", "assetID", "filtered_samples"

May have to be modified (Depending on your actual situation) : 

In [ ]:
#### If you are trying to use geemap in coutries where Gooogle Services are blocked (e.g., China), 
#### you will need a VPN,then replace "10809" with your "proxy port number"to connect to Earth Engine servers.
#### Otherwise, you might encounter a connection timeout issue.

import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [ ]:
#### Initializing GEE

import geemap
import ee
Map=geemap.Map()
Map

In [ ]:
#### Setting the boundaries of the study area.
#### Format: ee.Geometry.Rectangle(minLng, minLat, maxLng, maxLat)

roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
#### Defining variables
region = 'wuhan'

#### Calls for elevation data, which he has downloaded to GEE's Assets in "00--preparation.ipynb"
elevation = ee.Image("users/311605001111/hillshade_wuhan")

#### Visualisation parameters for Landsat8 images
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

# Define the relevant functions

In [ ]:
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi)

def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image_shadow = img.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True).remap([0,1],[1,0]))
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    image_outlier = img.updateMask(bands.gt(0).rename('outlier'))
    return ee.ImageCollection([image_cloud,image_shadow,image_outlier]).sum()

In [ ]:
## img refers to an image that has been indexed but not cloud-masked
def classified_image(img):
    image = maskSR(img).select(bands).classify(trainedClassifier).eq(1).remap([0,1],[1,2]).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,image]).sum()
    return class_image.set({'system:id':img.get('system:id')}).set({'CLOUD_COVER':img.get('CLOUD_COVER')})

# Water occurrence

You can view the download progress in "Google earth engine->Code Editor->Tasks"

"filtered_samples" denotes the training sample set downloaded to GEE's Assets in "01--Automatic training sample construction.ipynb",which needs to be adapted to your situation.

In [ ]:
# TIME = [2000, 2005, 2010, 2015, 2020]
TIME = [2020]
for year in range TIME:
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    landsat_image = l8.merge(l7).merge(l5).map(water_index)
    print('Total number of images： ',landsat_image.size().getInfo())
    
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    filtered_samples = ee.FeatureCollection('users/311605001111/' + region + '/' + region + '_9920')
    trainedClassifier = ee.Classifier.smileRandomForest(150).train(filtered_samples,label,bands)
    
    #### Calculation of water occurrence
    class_image = landsat_image.map(classified_image)
    waterPixel = class_image.map(lambda i : i.select('waterclass').eq(2)).sum().rename('waterNumber')
    validPixel = class_image.map(lambda i : i.select('waterclass').gt(0)).sum().rename('validNumber')
    waterfrequency = waterPixel.divide(validPixel).rename('frequency')
    # Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")
        
    #### The data will be stored in GEE's Assets
    ## "dataset_id" indicates the file name; "assetID" indicates the file path in GEE's Assets.
    ## They need to be modified to suit your situation
    dataset_id = region + '_WF_' + str(year)
    assetID = 'users/311605001111/WF_uncorrected/' + dataset_id

    task = ee.batch.Export.image.toAsset(**{
        'image': waterfrequency,
        'description': dataset_id,
        'assetId': assetID,
        'scale': 30,
        'region': roi,
        'maxPixels': 1e13,
    })
    task.start()

# Appendix--download

By default, all processing results are saved to GEE's Assets. 

If required, they can also be downloaded to a local disk or Google Drive

## Download to Google Drive

In [ ]:
#### Calling the image data stored in GEE's Assets. 
## "data_id" indicates the image id stored in GEE's Assets

data_id = 'users/311605001111/WF_uncorrected/wuhan_WF_2020'
waterfrequency = ee.Image(data_id)

In [ ]:
#### The data will be stored in Google Drive
## "dataset_id" indicates the file name; "folder" indicates the file path in Google Drive.
## They need to be modified to suit your situation

year = 2020
region = 'wuhan'
dataset_id = region + '_WF_uncorrected_' + str(year)
folder = region + '_WF_uncorrected'

task = ee.batch.Export.image.toDrive(**{
    'image': waterfrequency,
    'description': dataset_id,
    'folder': folder,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

## Download to local disk

In [ ]:
#### Calling the image data stored in GEE's Assets. 
## "data_id" indicates the image id stored in GEE's Assets

data_id = 'users/311605001111/WF_uncorrected/wuhan_WF_2020'
waterfrequency = ee.Image(data_id)

In [ ]:
#### The data will be stored in local dish
## "dataset_id" indicates the file name; "folder indicates the file path in local dish.
## For example, the path for this experiment is "D:/wuhan_WF_uncorrected/wuhan_WF_uncorrected_2020.tif"
## They need to be modified to suit your situation

year = 2020
region = 'wuhan'
folder = 'D:/' + region + '_WF_uncorrected/'+ region + '_WF_uncorrected_' + str(year) + '.tif'

task = geemap.ee_export_image(**{
    'ee_object' : waterfrequency,
    'filename' : folder, 
    'scale' : 30, 
    'region' : roi, 
    'file_per_band' : False        
})
task